In [ ]:
from pip._internal import main
main(['install','mysql-connector-python-rf'])
import mysql.connector
import requests
from bs4 import BeautifulSoup

In [58]:
base_url = 'https://247sports.com/'
url = base_url + "Season/2021-Football/CollegeTeamTalentComposite/?Conference=";
conferences = ['ACC','Big-12','Big-Ten','Pac-12','SEC','AAC']

In [61]:
# DATA
data = get_teams_general_info()

In [60]:
def parse_team_info(soup):
    a = soup.find('a', {'class':'rankings-page__name-link'})
    teamName = a.string.strip()
    team_breakdown_url = a['href'] if a.has_attr('href') else None
    logo_container = soup.find('img')
    logo_url = logo_container['data-src'] if logo_container.has_attr('data-src') else None
    if logo_url is not None:
        logo_url = logo_url.split('?')[0]
    
    team_data = {'name':teamName,'team_url' : team_breakdown_url, 'logo_url' : logo_url}
    return team_data

def get_teams_general_info():
    data = {}
    for conference in conferences:
        conference_url = url + conference
        html = requests.get(conference_url, headers={"User-Agent": "Requests"}).content
        soup = BeautifulSoup(html, 'html.parser')
        team_info_containers = soup.find_all('li', {'class' : 'rankings-page__list-item'})
        for team_container in team_info_containers:
            team_data = parse_team_info(team_container)
            team_data['conference'] = conference
            data[team_data['name']] = team_data
    return data

def get_team_talent_data(team_data, team_name):
    team_url = base_url + team_data['team_url']
    html = requests.get(team_url, headers={"User-Agent": "Requests"}).content
    soup = BeautifulSoup(html, 'html.parser')
    table_body = soup.find('tbody')
    # extract team members
    containers = table_body.find_all('td')
    roster = []
    for container in containers:
        a = container.find('a')
        name = a.string if a else container.find('span').string
        roster.append(name)
    
    players = []
    # extract talent info
    stats_container = soup.find('div', {'class' : 'scroll-table-container'})
    table_body = stats_container.find('tbody')
    rows = table_body.find_all('tr')
    if len(rows) != len(roster):
        print("##### error: mismach roster and stats")
    else:
        for i in range(len(rows)):
            name = roster[i]
            row = rows[i]
            stats = row.find_all('td')
            position = stats[1].string
            height = stats[2].string
            weight = stats[3].string
            year = stats[4].string
            stars = row.find_all('span', {'class':'icon-starsolid'})
            player = {'name':name, 'team': team_name, 'position' : position, 'height' : height, 'weight' : weight, 'year' : year, 'stars' : len(stars)}
            players.append(player)
    return players

In [ ]:
# playerDB = []
# for team_name in data:
#     print("fetching data for: " + team_name)
#     players = get_team_talent_data(data[team_name], team_name)
#     playerDB.extend(players)

In [ ]:
def filterByTeam(players, team):
    filtered = filter(lambda player: player["team"] == team, players)
    return filtered

def filterByPosition(players, position):
    filtered = filter(lambda player: player['position'] == position, players)
    return filtered

def filterByTeamAndPosition(players, team, position):
    filtered = filter(lambda player: player['position'] == position and player['team'] == team, players)
    return filtered

filtered = filterByTeamAndPosition(playerDB, 'Baylor', 'WR');
for player in filtered:
    print(player)
    break



In [53]:
connection = mysql.connector.connect(user='cfbuser', password='cfbuser',
                              host='127.0.0.1',
                              database='cfbdb')
mycursor = connection.cursor()

In [54]:
# select 
mycursor.execute("SELECT * FROM conference")
results = mycursor.fetchall()
print(results)

[('AAC',), ('ACC',), ('Big-12',), ('Big-Ten',), ('Pac-12',), ('SEC',)]


In [51]:
# Insert conferences into conference table
for name in conferences:
    sql = (
       "INSERT IGNORE INTO conference(name)"
       "VALUES (%s)"
    )
    data = (name,)
    mycursor.execute(sql, data)
    connection.commit()

In [71]:
# Insert team data into db

# create list of values for inserts
teams = []
for team_name in data:
    info = data[team_name]
    item = (info['name'], info['team_url'], info['logo_url'], info['conference'])
    teams.append(item)

In [72]:
# insert teams into db
sql = """INSERT INTO team (name, team_url, logo_url, conference_name) 
                           VALUES (%s, %s, %s, %s) """


cursor.executemany(sql, teams)
connection.commit()
print(cursor.rowcount, "Record inserted successfully into Laptop table")

[('Clemson',
  '/Team/Clemson-3/Roster',
  'https://s3media.247sports.com/Uploads/Assets/565/682/4682565.png',
  'ACC'),
 ('Miami',
  '/Team/Miami-13/Roster',
  'https://s3media.247sports.com/Uploads/Assets/201/648/4648201.png',
  'ACC'),
 ('North Carolina',
  '/Team/North-Carolina-15/Roster',
  'https://s3media.247sports.com/Uploads/Assets/602/120/8120602.png',
  'ACC'),
 ('Florida State',
  '/Team/Florida-State-7/Roster',
  'https://s3media.247sports.com/Uploads/Assets/414/682/4682414.png',
  'ACC'),
 ('Georgia Tech',
  '/Team/Georgia-Tech-9/Roster',
  'https://s3media.247sports.com/Uploads/Assets/197/648/4648197.png',
  'ACC'),
 ('NC State',
  '/Team/NC-State-17/Roster',
  'https://s3media.247sports.com/Uploads/Assets/728/975/7975728.png',
  'ACC'),
 ('Pittsburgh',
  '/Team/Pittsburgh-460/Roster',
  'https://s3media.247sports.com/Uploads/Assets/109/167/9167109.png',
  'ACC'),
 ('Virginia Tech',
  '/Team/Virginia-Tech-21/Roster',
  'https://s3media.247sports.com/Uploads/Assets/212/64

In [52]:
connection.close()